In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('../city_data.csv', delimiter='|')

new_header = data.iloc[0]
data = data[1:]
data.columns = new_header

cities = []
states = []
for i in range(len(data)):
    if ',' in data.iloc[i]["City"]:
        city_and_state = data.iloc[i]["City"].split(",")
    elif '.' in data.iloc[i]["City"]:
        city_and_state = data.iloc[i]["City"].split(".")
    else:
        city_and_state = data.iloc[i]["City"].split(";")
         
    cities.append(city_and_state[0])
    if len(city_and_state) > 1:
        states.append(city_and_state[1])
    else:
        states.append("")

data.drop(columns=['City'], inplace=True)
data.insert(0, 'City', cities)
data.insert(1, 'Country', states)

data.drop(columns=['Average Price Groceries'], inplace=True)

data.drop_duplicates(inplace=True)

for col in data.columns:
    try:
        data[col] = pd.to_numeric(data[col])
    except:
        pass  # keep text columns (City, Country) as is


numeric_cols = data.select_dtypes(include=[np.number]).columns
scaler = StandardScaler()
scaled = scaler.fit_transform(data[numeric_cols])

imputer = KNNImputer(n_neighbors=7)
imputed_scaled = imputer.fit_transform(scaled)

# Inverse scale back
imputed = scaler.inverse_transform(imputed_scaled)

# Put back into DataFrame
data[numeric_cols] = imputed


pd.set_option('display.max_rows', None)     # show all rows
pd.set_option('display.max_columns', None)  # show all columns

data = data[['Country', 'City', 'Average Cost of Living', 'Average Monthly Salary']]
data['Country'] = data['Country'].str.strip()
data


#https://www.kaggle.com/datasets/gianinamariapetrascu/gender-pay-gap-europe-2010-2021/data

pay_gap_data_all = pd.read_csv('../pay_gap_Europe.csv', sep=',')
pay_gap_data = pay_gap_data_all[pay_gap_data_all['Year'] == 2021]
#pay_gap_data.fillna(0, inplace=True)
pay_gap_data['Avg pay gap in 2021'] = pay_gap_data[['Industry', 'Business',
       'Mining', 'Manufacturing', 'Electricity_supply', 'Water_supply',
       'Construction', 'Retail trade', 'Transportation', 'Accommodation',
       'Information', 'Financial', 'Real estate ', 'Professional_scientific',
       'Administrative', 'Public_administration', 'Education', 'Human_health',
       'Arts', 'Other']].mean(axis=1)
pay_gap_data = pay_gap_data[['Country', 'Avg pay gap in 2021']]
pay_gap_data['Country'] = pay_gap_data['Country'].str.strip()

full_data = pd.merge(data, pay_gap_data, how='left', on='Country')
full_data.stack()
full_data

pay_gap_data_all.sort_values(['Country', 'Year'], ascending=(False,True), inplace=True)
pay_gap_data_all['Avg pay gap'] = pay_gap_data_all[['Industry', 'Business',
       'Mining', 'Manufacturing', 'Electricity_supply', 'Water_supply',
       'Construction', 'Retail trade', 'Transportation', 'Accommodation',
       'Information', 'Financial', 'Real estate ', 'Professional_scientific',
       'Administrative', 'Public_administration', 'Education', 'Human_health',
       'Arts', 'Other']].mean(axis=1)

#https://www.kaggle.com/datasets/jegazhu/cheapest-cost-to-study-in-europe-2023
students_cost_data = pd.read_excel('../cheapest_countries_to_study_Europe_2023.xlsx')
full_data = pd.merge(full_data, students_cost_data, how='left', left_on='Country', right_on='country')
full_data.drop('country', axis=1, inplace=True)

C:\Users\yairt\AppData\Local\Temp\ipykernel_24440\4139340451.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pay_gap_data['Avg pay gap in 2021'] = pay_gap_data[['Industry', 'Business',


There are several dataframes suitable for performing the analysis - comparison between European cities/countries from financial standpoint.

In [16]:
#first dataframe with data gathered together.
full_data.head()

,Country,City,Average Cost of Living,Average Monthly Salary,Avg pay gap in 2021,total_yearly_living_costs_and_fees_ps,average_yearly_tuition_fees_ps,yearly_student_living_costs_ps
0,Austria,Vienna,2061.0,2500.0,17.078947,15324.0,1254.0,14071.0
1,Austria,Salzburg,2186.0,3200.0,17.078947,15324.0,1254.0,14071.0
2,Belgium,Brussels,1900.0,3350.0,5.705263,16286.0,3602.0,12684.0
3,Belgium,Antwerp,1953.0,2609.0,5.705263,16286.0,3602.0,12684.0
4,Belgium,Gent,1200.0,2400.0,5.705263,16286.0,3602.0,12684.0


In [17]:
#second dataframe with data about percentages of gender pay gaps by country.
pay_gap_data_all.head()

,Country,Year,GDP,Urban_population,Industry,Business,Mining,Manufacturing,Electricity_supply,Water_supply,Construction,Retail trade,Transportation,Accommodation,Information,Financial,Real estate,Professional_scientific,Administrative,Public_administration,Education,Human_health,Arts,Other,Avg pay gap
312,Switzerland,2010,57730,73.61,17.8,23.0,5.2,19.8,15.8,8.4,8.9,27.3,13.7,8.4,24.2,32.0,22.5,24.0,21.6,17.2,12.5,19.1,14.4,22.7,17.925
313,Switzerland,2011,58220,73.63,17.6,20.6,10.4,18.4,12.7,8.5,8.7,23.4,13.1,8.9,21.3,31.5,20.9,23.4,17.4,17.9,8.1,18.2,15.6,20.5,16.855
314,Switzerland,2012,58290,73.65,17.4,20.3,9.6,18.2,11.9,7.7,8.6,23.1,13.4,8.0,21.0,31.1,20.7,23.3,17.3,18.3,8.0,18.1,15.4,20.2,16.580
315,Switzerland,2013,58650,73.67,17.6,20.3,9.0,18.3,11.3,7.1,8.4,22.9,13.1,8.7,21.7,31.1,20.9,23.3,17.6,17.9,8.0,18.4,15.3,20.2,16.555
316,Switzerland,2014,59300,73.70,17.4,19.9,8.2,18.0,10.5,6.3,7.7,22.3,12.8,8.1,21.6,31.0,20.6,22.6,17.4,17.9,7.9,18.7,14.5,19.4,16.140


In [18]:
#third dataframe with data about financial cost for students by country. (this dataframe is contained within full_data)
students_cost_data.head()

,country,total_yearly_living_costs_and_fees_ps,average_yearly_tuition_fees_ps,yearly_student_living_costs_ps
0,Bulgaria,9020,1790,7230
1,Romania,10025,3019,7006
2,Portugal,10410,690,9720
3,Lithuania,10877,1122,9755
4,Slovakia,10996,1725,9271


In [19]:
#forth dataframe of Average hourly earnings of employees by sex and occupation
data4 = pd.read_csv('../bysexandoccupation.csv')
#subset only European countries
data4 = data4[data4['ref_area.label'].isin(full_data['Country'].tolist())]
data4.drop(['obs_status.label', 'note_classif.label', 'note_source.label'], axis=1, inplace=True)

avg_hourly_earning = pd.DataFrame(data4.groupby(by=['ref_area.label', 'classif1.label'])['obs_value'].mean())
avg_hourly_earning.head()


obs_value
ref_area.label classif1.label                                               
Austria        Occupation (ISCO-08): 0. Armed forces occupations   23.842000
               Occupation (ISCO-08): 1. Managers                   41.394889
               Occupation (ISCO-08): 2. Professionals              29.343578
               Occupation (ISCO-08): 3. Technicians and associ...  24.156733
               Occupation (ISCO-08): 4. Clerical support workers   19.913044

In [ ]:
#https://www.kaggle.com/datasets/mvieira101/global-cost-of-living?utm_source=chatgpt.com
#fifth dataframe of cost of living

cost_of_living_data = pd.read_csv('../cost-of-living.csv')
cost_of_living_data = cost_of_living_data.iloc[:, 1:]
columns = [
    "city", "country",
    "meal_inexpensive_restaurant", "meal_for_2", "mcmeal_mcdonalds",
    "domestic_beer_restaurant", "imported_beer_restaurant", "cappuccino",
    "coke_pepsi", "water_restaurant",
    "milk_1L", "bread_500g", "rice_1kg", "eggs_12", "cheese_1kg", "chicken_1kg",
    "beef_1kg", "apples_1kg", "banana_1kg", "oranges_1kg", "tomato_1kg",
    "potato_1kg", "onion_1kg", "lettuce_1head",
    "water_1_5L", "wine_midrange", "beer_domestic_market", "beer_imported_market",
    "cigarettes_pack", "one_way_ticket", "monthly_pass", "taxi_start",
    "taxi_1km", "taxi_1hour", "gasoline_1L",
    "vw_golf_new", "toyota_corolla_new", "utilities_85m2",
    "mobile_tariff_min", "internet_unlimited", "fitness_club", "tennis_court",
    "cinema", "preschool_monthly", "intl_primary_yearly",
    "jeans", "dress", "nike_shoes", "leather_shoes",
    "apartment_1br_centre", "apartment_1br_outside",
    "apartment_3br_centre", "apartment_3br_outside",
    "price_m2_centre", "price_m2_outside", "avg_monthly_salary",
    "mortgage_interest_rate", "data_quality"
]
cost_of_living_data.columns = columns
cost_of_living_data = cost_of_living_data[cost_of_living_data['country'].isin(full_data['Country'])]
cost_of_living_data.reset_index(inplace=True)

cost_of_living_data["avg_food_cost"] = cost_of_living_data[[
    "milk_1L", "bread_500g", "rice_1kg", "eggs_12",
    "cheese_1kg", "chicken_1kg", "beef_1kg",
    "apples_1kg", "banana_1kg", "tomato_1kg", "potato_1kg"
]].mean(axis=1)

cost_of_living_data["avg_entertainment_cost"] = cost_of_living_data[[
    "meal_inexpensive_restaurant", "meal_for_2", "cappuccino", "cinema", "beer_domestic_market"
]].mean(axis=1)

cost_of_living_data["avg_housing_cost"] = cost_of_living_data[[
    "apartment_1br_centre", "apartment_1br_outside",
    "apartment_3br_centre", "apartment_3br_outside", "utilities_85m2"
]].mean(axis=1)

cost_of_living_data["avg_transport_cost"] = cost_of_living_data[[
    "monthly_pass", "gasoline_1L", "taxi_1km"
]].mean(axis=1)


cost_of_living_data_minimized = cost_of_living_data[['country', 'city', "avg_food_cost", "avg_entertainment_cost", "avg_housing_cost",\
                                                     "avg_transport_cost"]]

cost_of_living_data_minimized.head()


,country,city,avg_food_cost,avg_entertainment_cost,avg_housing_cost,avg_transport_cost
0,France,Paris,6.544545,19.248,1497.684,27.300000
1,United Kingdom,London,3.913636,24.078,2175.196,61.443333
2,Spain,Madrid,3.855455,15.394,1020.256,19.926667
3,Spain,Barcelona,4.134545,17.934,1045.140,15.216667
4,Germany,Berlin,4.899091,18.336,1339.340,30.886667
